In [1]:
import pandas as pd
import bqplot
import numpy as np
import ipywidgets

In [2]:
data = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv")
data["County"] = data["County"].fillna("unknown")

In [3]:
data.isnull().any()

Agency Name                False
Location Name              False
Address                     True
City                       False
Zip code                   False
County                     False
Congress Dist              False
Congressional Full Name     True
Rep Dist                   False
Rep Full Name               True
Senate Dist                False
Senator Full Name           True
Bldg Status                False
Year Acquired              False
Year Constructed           False
Square Footage             False
Total Floors               False
Floors Above Grade         False
Floors Below Grade         False
Usage Description          False
Usage Description 2         True
Usage Description 3         True
dtype: bool

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Agency Name              8862 non-null   object
 1   Location Name            8862 non-null   object
 2   Address                  8811 non-null   object
 3   City                     8862 non-null   object
 4   Zip code                 8862 non-null   int64 
 5   County                   8862 non-null   object
 6   Congress Dist            8862 non-null   int64 
 7   Congressional Full Name  8699 non-null   object
 8   Rep Dist                 8862 non-null   int64 
 9   Rep Full Name            8839 non-null   object
 10  Senate Dist              8862 non-null   int64 
 11  Senator Full Name        8839 non-null   object
 12  Bldg Status              8862 non-null   object
 13  Year Acquired            8862 non-null   int64 
 14  Year Constructed         8862 non-null  

In [5]:
data_right = pd.DataFrame(data.groupby(["County","Agency Name"])["Square Footage"].mean())
data_right = data_right.reset_index()

In [6]:
df_heatmap = pd.pivot_table(data=data,values="Square Footage",index="Agency Name",columns="County",aggfunc=np.mean, fill_value=1)
df_heatmap = np.log10(df_heatmap)

In [7]:
rows_name = list(df_heatmap.index)
columns_name = list(df_heatmap.columns)

In [8]:
# df_heatmap_array = np.array(df_heatmap)

i,j = 6,15
mask = (data['Agency Name'] == df_heatmap.index[i]) & \
            (data['County'] == df_heatmap.columns[j])

x_scs = bqplot.LinearScale(min=1753,max=2020)
y_scs = bqplot.LinearScale(max=1200000)

x_axs = bqplot.Axis(label='Year', scale=x_scs)
y_axs = bqplot.Axis(label='total square footage acquired that year', scale=y_scs, 
                   orientation = 'vertical')

buildings_subset = data[mask]
    # grab years and sqft from pandas
grouped = buildings_subset.groupby("Year Acquired")["Square Footage"].sum()
years = grouped.index
sqft = grouped.values
bar = bqplot.Bars(x=years,y = sqft,scales={'x':x_scs, 'y':y_scs})

fig_dur = bqplot.Figure(marks=[bar], axes=[x_axs,y_axs])
fig_dur

Figure(axes=[Axis(label='Year', scale=LinearScale(max=2020.0, min=1753.0)), Axis(label='total square footage a…

In [9]:
col_sc = bqplot.ColorScale(scheme="viridis")
x_sc = bqplot.OrdinalScale() # categorical data
y_sc = bqplot.OrdinalScale()

ax_col = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
ax_x = bqplot.Axis(scale=x_sc, tick_rotate=45,label="Agency Name")
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical',label="County")

heat_map = bqplot.GridHeatMap(color=df_heatmap,
                              row = rows_name,
                              column = columns_name,
                             scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'}, 
                             anchor_style={'fill':'green'})

mySelectedLabel = ipywidgets.Label()

def on_selected(change):
    i,j = change['owner'].selected[0]
    mask = (data['Agency Name'] == df_heatmap.index[i]) & \
            (data['County'] == df_heatmap.columns[j])
    
    # grab buildings subset dataframe
    buildings_subset = data[mask]
    # grab years and sqft from pandas
    grouped = buildings_subset.groupby("Year Acquired")["Square Footage"].sum()
    years = grouped.index
    sqft = grouped.values
    bar.x = years
    bar.y = sqft
    mySelectedLabel.value = 'location = ' + str(i)+ ","+ str(j)

heat_map.observe(on_selected, 'selected')

fig = bqplot.Figure(marks=[heat_map], axes=[ax_col, ax_x, ax_y],title="Grid Heat Map")
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='viridis'), side='right'), Axis(label='…

In [10]:
fig.layout.min_width='500px'
fig_dur.layout.min_width = '500px'
figures = ipywidgets.HBox([fig, fig_dur])

myDashboard = ipywidgets.VBox([mySelectedLabel, figures])
myDashboard

Things to think about:

**Can you keep the x and y ranges static on the bar plot?**

Defintely, we could set the values of max and min of bqplot.LinearScale() to keep the x and y ranges static on the bar plot.

**Can you change the style?**

Yes, bqplot.Bars contains plenty attributes which control the stype of the barplot so that we could change the scheme by changing attributes like "colors", "stroke" ... or setting bqplot.ColorScale.

In your write-up please address:

**Any data transformations or rescalings you did for your plot**

When I build the pivot_table to prepare the heatmap, I transform the mean of total square footage into log10 based values.


**How you dealt with NaN's (empty entries) in your dataset**

First, check which columns contain NAN by typing ***data.isnull().any()***. "Agency Name", "Year Acquired" and "Square Footage" does not contain any NAN. Set NAN in "County" to be "unknown" and set the fill_value of pivot_table to be 0 to processing the NAN.

**Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.**

First, I set the scheme of Grid Heat Map into "viridis" format. I have try plenty of schemes including "RdPu", but they all hard to read. 

Next, set the tick_rotate of x_axis for GridHeatMap into 45 degree in order to split each x element. But still not perfect.

Finally, adjust the width of each plot in the dashboard(fig.layout.min_width='500px', fig_dur.layout.min_width = '500px') to make the plots more readable.


